In [2]:
from binance import Client
import requests
from dotenv import load_dotenv
import os
import ta
import pandas as pd
import time

In [3]:
load_dotenv()

True

In [5]:
# api_key = os.getenv("BINANCE_API_KEY")
# api_secret = os.getenv("BINANCE_SECRET_KEY")

In [6]:
client = Client(api_key, api_secret)

In [9]:
def getminutedata(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval,  lookback + 'm ago UTC'))
    frame = frame.iloc[:, :6]
    frame.columns = ['Time','Open', 'High','Low','Close','Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

In [14]:
df = getminutedata('ETHUSDT', '1m','100')

In [33]:
def applytechnicals(df):
    df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Low, df.Close, window=14, smooth1 = 3, smooth2 = 3)
    df['%D'] = df['%K'].rolling(3).mean()
    df['rsi'] = ta.momentum.rsi(df.Close, window=14)
    df['macd'] = ta.trend.macd_diff(df.Close)
    df.dropna(inplace=True)


In [34]:
applytechnicals(df)

TypeError: stochrsi_k() got multiple values for argument 'window'

In [ ]:
class Signals:
    
    def __init__(self,df, lags):
        self.df = df
        self.lags = lags
        
    def gettrigger(self):
        dfx = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df['%k'].shift(1) < 20) & (self.df['%D'].shift(i) <20)
            dfx = dfx.append(mask, ignore_index=True)
        return dfx.sum(axis=0)
    
    def decide(self):
        self.df['trigger'] = np.where(self.gettrigger(), 1, 0)
        self.df['Buy'] = np.where((self.df.trigger) &
                        (self.df['%K'].between(20,80)) & (self.df['%D'].between(20,80)) & 
                        (self.df.rsi >50) & (self.df.macd > 0))
                                 
                                 
                                 
        

In [ ]:
instance = Signals(df, 5)

In [ ]:
instance.decide()

In [ ]:
df[df.Buy == 1]

In [ ]:
def strategy(pair, qty, open_position=False):
    df = getminutedata(pair, '1m', '100')
    applytechnicals(df)
    instance = Signals(df, 25)
    instance.decide()
    print(f'current close is' +str(df.Close.iloc[-1]))
    if df.Buy.iloc[-1]:
        order = client.create_order(symbol=pair, side='BUY', type='MARKET', quantity=qty)
        print(order)
        buyprice =float(order['fills'][0]['price'])
        open_position = True
    while open_position:
        time.sleep(0.5)
        df = getminutedata(pair, '1m', '2')
        print(f'current Close '+ str(df.Close.iloc[-1]))
        print(f'current Target '+ str(buyprice*1.005))
        print(f'current Stop is '+ str(buyprice * 0.995))
        if df.Close[-1] <= buyprice *0.995 or df.Close >= 1.005 * buyprice:
            order = client.create_order(symbol=pair, side='BUY', type='MARKET', quantity=qty)
            print(order)
            break
            

In [ ]:
while True:
    strategy('ETHUSDT', 50)
    time.sleep(0.5)